In [1]:
from ROOT import TH1D, TCanvas, TF1, std
from pyLCIO import EVENT, UTIL, IOIMPL, IMPL
#from UTIL import CellIDDecoder
import matplotlib.pyplot as plt
import numpy as np
import h5py
import matplotlib as mpl
import math
plt.rcParams.update({'font.size': 17})

from functions import Esumhit
from functions import CellIDDecoder

import uncertainties
#from uncertainties import unumpy

def getTotE(data, xbins, ybins, layers):
    data = np.reshape(data,[-1, layers*xbins*ybins])
    etot_arr = np.sum(data, axis=(1))
    return etot_arr




Welcome to JupyROOT 6.22/02
Loading LCIO ROOT dictionaries ...


In [2]:
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)


color_list = []
color_list.append('dimgrey')
color_list.append('black')


linewidth_list = []
linewidth_list.append(1.5)
linewidth_list.append(2)

    
linestyle_list = []
linestyle_list.append('-')
linestyle_list.append('--')

fillcolor_list = []
fillcolor_list.append('lightgrey')
fillcolor_list.append('red')


def plt_geant4(data_real, data_real2, energy_center, maxE, minE, bins, xtitle,save_title):
    figSE = plt.figure(figsize=(8,8))
    axSE = figSE.add_subplot(1,1,1)
    lightblue = (0.1, 0.1, 0.9, 0.3)
    
   
    pSEa = axSE.hist(data_real, bins=bins, range=[minE, maxE], density=None, edgecolor=color_list[0],
    #weights=np.ones_like(data_real)/(float(len(data_real))),  
                   label = "orig", linewidth=linewidth_list[0],color=fillcolor_list[0],
                   histtype='stepfilled')

   

    pSpnEb = axSE.hist(data_real2, bins=pSEa[1], range=None, density=None, edgecolor=color_list[1],
    #weights=np.ones_like(data_fake)/(float(len(data_fake))), , 
                   label = "orig", linewidth=linewidth_list[1], linestyle=linestyle_list[1],
                   histtype='step')

 
    
    axSE.set_xlabel(xtitle, family='serif')
    axSE.set_xlim([minE, maxE])
    #axSE.set_ylim([0, 0.28])

  
    
    posX = 0.02
    axSE.text(posX,
            0.97,
            '{:d} GeV Photons'.format(energy_center), horizontalalignment='left',verticalalignment='top', 
             transform=axSE.transAxes)

    axSE.text(posX, 0.65, 'Geant4 (Pure LCIO hits)', horizontalalignment='left',verticalalignment='top', 
             transform=axSE.transAxes, color = color_list[0])
    axSE.text(posX, 0.60, 'Geant4 (Hdf5 file 30x30x40)', horizontalalignment='left',verticalalignment='top', 
             transform=axSE.transAxes, color = color_list[1])
    
    plt.subplots_adjust(left=0.18, right=0.95, top=0.95, bottom=0.18)
    #plt.yscale('log')
    figSE.patch.set_facecolor('white')

    plt.savefig(save_title+ str(energy_center)+"_leak.png")
    
    
    
def plt_scatter(x_data, y_data, y_err, y_data_lcio, y_err_lcio):
    figStr = plt.figure(figsize=(6,6))
    axStr = figStr.add_subplot(1,1,1)
    
    lightblue = (0.1, 0.1, 0.9, 0.3)
    markersize1=10
    markersize2=10
    

    pStra = axStr.errorbar(x=x_data, y=y_data, xerr=None, yerr=y_err, 
                               capsize=4, marker='x', linestyle='-', 
                               linewidth=2, color='red')
    pStrb = axStr.errorbar(x=x_data, y=y_data_lcio, xerr=None, yerr=y_err_lcio, 
                               capsize=4, marker='x', linestyle='dashed', 
                               linewidth=2, color='lightcoral')
    
    #axStr.text(0.70, 0.93, 'Hdf5', horizontalalignment='left',verticalalignment='top', 
    #         transform=axStr.transAxes, color='black')
    
    axStr.text(0.60, 0.93, 'LCIO-Nominal', horizontalalignment='left',verticalalignment='top', 
             transform=axStr.transAxes, color='red')
    
    axStr.text(0.60, 0.87, 'LCIO-corrected', horizontalalignment='left',verticalalignment='top', 
             transform=axStr.transAxes, color='lightcoral')
    
    
    axStr.xaxis.set_minor_locator(MultipleLocator(0.1))
    axStr.xaxis.set_major_locator(MultipleLocator(0.2))
    
    axStr.patch.set_facecolor('white')
    figStr.patch.set_facecolor('white')
    
    #plt.yscale('log')
    
    axStr.set_ylabel('< E-sum >', family='serif')
    axStr.set_xlabel('Angle [rad]', family='serif')
    axStr.set_xlim([0.4, 1.7])
    axStr.set_ylim([415, 550])
    plt.savefig("plots/leakDiff.png")
    
def plt_scatter_ratio(x_data, y_data, y_lcio):
    figStr = plt.figure(figsize=(6,6))
    axStr = figStr.add_subplot(1,1,1)
    
    lightblue = (0.1, 0.1, 0.9, 0.3)
    markersize1=10
    markersize2=10
    
    ratio = y_data / y_lcio

    ## for correlated uncertainties
    corr_unc = []
    for i in range(len(y_data)):
        a, b = uncertainties.correlated_values_norm(
            [(ydata[i].n, ydata[i].s), (ydata_lcio[i].n, ydata_lcio[i].s)], [[1, 1], [1, 1]])
        tmp = a/b
        corr_unc.append(tmp.s) 
    
    pStra = axStr.errorbar(x=x_data, y=unumpy.nominal_values(ratio), xerr=None, yerr=corr_unc, 
                               capsize=4, marker='x', linestyle='-', 
                               linewidth=2, color='black')
   
    
    #axStr.text(0.70, 0.93, 'Ratio of Hdf5', horizontalalignment='left',verticalalignment='top', 
    #         transform=axStr.transAxes, color='black')
    
    
    
    axStr.xaxis.set_minor_locator(MultipleLocator(0.1))
    axStr.xaxis.set_major_locator(MultipleLocator(0.2))
    
    
    axStr.yaxis.set_minor_locator(MultipleLocator(0.01))
    axStr.yaxis.set_major_locator(MultipleLocator(0.05))
    
    axStr.patch.set_facecolor('white')
    figStr.patch.set_facecolor('white')
    
    #plt.yscale('log')
    
    axStr.set_ylabel('Hdf5 / LCIO', family='serif')
    axStr.set_xlabel('Angle [rad]', family='serif')
    axStr.set_xlim([0.4, 1.7])
    axStr.set_ylim([0.94, 1.05])
    plt.savefig("plots/leakDiff_ratio.png")
      


In [ ]:
g4_sim = Esumhit("40_deg/photon-40.slcio_REC.slcio", 2000, "EcalBarrelCollection" )

In [ ]:
f = h5py.File('40_deg/photon20GeV-40deg.hdf5','r')
layers = f['ecal/layers']
showers = layers[:]

In [ ]:
g4_sim_box = getTotE(showers,30,40,30)

In [ ]:
plt_geant4(g4_sim*1000, g4_sim_box, 20, 650, 100, bins=50, xtitle="Energy Sum [MeV]",save_title="plots/leak-40deg-comp")

## Let's try for diffential leak (as a function of angle)

In [ ]:
f = h5py.File('photon20GeV-30x40-1.hdf5','r')
#f = h5py.File('40_deg/photon20GeV-40deg.hdf5','r')
layers = f['ecal/layers']
ang = f['ecal/theta']
showers = layers[:]
angles = ang[:]

In [ ]:
showers.shape

In [ ]:
def Esumhit_perTheta(fName, maxEvt, collection, thMin, thMax):
    reader = IOIMPL.LCFactory.getInstance().createLCReader()
    reader.open( fName )
    
    esuml = []

    nEvt = 0

    for evt in reader:
        nEvt += 1
        if nEvt > maxEvt:
                break
        
        mcparticle = evt.getCollection("MCParticle")  
       
        for enr in mcparticle:
            E = enr.getEnergy()
            ## calculate polar angle theta
            if enr.getGeneratorStatus() != 1.0: continue
            pVec = enr.getMomentum()
            theta = math.pi/2.00 - math.atan(pVec[2]/pVec[1])
             
        
        if (theta < thMax) & (theta >= thMin):
            ecalBarrel = evt.getCollection(collection)    
            esum = 0.0
            for hit in ecalBarrel:
                e = hit.getEnergy()*1000.00  ## convert to MeV 
                esum += e

            esuml.append(esum)
        
        
    esumnp = np.asarray(esuml)
    
    
    return esumnp


In [ ]:
x_data = []
y_data = []
y_err = []


lcio_y_data = []
lcio_y_err = []

for i in range(30,90,5):
    rad = i*math.pi/180
    radnn = (i+5)*math.pi/180
    
    print (rad, radnn)
    ## hdf5 guys
    idx = np.where( (angles <= radnn) & (angles > rad))
    x_data.append((rad + radnn)/2)
    sh = showers[idx[0]]
    esum = getTotE(sh,30,40,30)
    esum = np.nan_to_num(esum)
    #if esum.mean() == 0.0: continue
    y_data.append(esum.mean())
    y_err.append(esum.std() / math.sqrt(len(esum)))

    ### LCIO guys
    esum_lcio = Esumhit_perTheta("photon-1.slcio_REC.slcio", 5000, "EcalBarrelCollection", rad, radnn)
    #esum_lcio = Esumhit_perTheta("40_deg/photon-40.slcio_REC.slcio", 2000, "EcalBarrelCollection", rad, radnn)
    
    #if esum_lcio.mean() == 0.0: continue
    lcio_y_data.append(esum_lcio.mean())
    lcio_y_err.append(esum_lcio.std() / math.sqrt(len(esum_lcio)))
    

In [ ]:
plt_scatter(x_data, y_data, y_err, lcio_y_data, lcio_y_err)

In [ ]:
ydata = unumpy.uarray(y_data, y_err) 
ydata_lcio = unumpy.uarray(lcio_y_data, lcio_y_err)

In [ ]:
plt_scatter_ratio(x_data, ydata, ydata_lcio)

## Corrected files

In [ ]:
## multiply last 10 layers with 2 (to correct for different sampling fraction)
def esum_corr_perTheta(fName, maxEvt, collection, thMin, thMax):
    reader = IOIMPL.LCFactory.getInstance().createLCReader()
    reader.open( fName )
    

    
    nEvt = 0

    esumL = []
    for evt in reader:
        nEvt += 1
        if nEvt > maxEvt:
                break
        
        mcparticle = evt.getCollection("MCParticle")  
       
        for enr in mcparticle:
            E = enr.getEnergy()
            ## calculate polar angle theta
            if enr.getGeneratorStatus() != 1.0: continue
            pVec = enr.getMomentum()
            theta = math.pi/2.00 - math.atan(pVec[2]/pVec[1])
             
                
        
        
        ecalBarrel = evt.getCollection(collection)
        cellIDString = ecalBarrel.getParameters().getStringVal("CellIDEncoding")
        decoder = CellIDDecoder( cellIDString ) 
        
        if (theta < thMax) & (theta >= thMin):
            esum = 0.0
            for hit in ecalBarrel:
                l = decoder.layer( hit.getCellID0() ) 
                e = hit.getEnergy()*1000.0 ## convert to MeV 
                #print ("Energy:", hit.getEnergy(), " Cell ID0:", hit.getCellID0(), " layer: ", decoder.layer( hit.getCellID0() )) 
                if l < 21:
                    esum = esum + e 
                elif l >= 21:
                    esum = esum + e*2

            esumL.append(esum)
        
        
    esumnp = np.asarray(esumL)
    
    return esumnp
    

In [ ]:
x_data = []
lcio_y_data = []
lcio_y_err = []


lcio_y_corrdata = []
lcio_y_correrr = []

for i in range(30,90,5):
    rad = i*math.pi/180
    radnn = (i+5)*math.pi/180
    
    x_data.append((rad + radnn)/2)
    print (rad, radnn)
    ## corrected LCIO guys
    esum_lcio_corr = esum_corr_perTheta("photon-1.slcio_REC.slcio", 5000, "EcalBarrelCollection", rad, radnn)
    lcio_y_corrdata.append(esum_lcio_corr.mean())
    lcio_y_correrr.append(esum_lcio_corr.std() / math.sqrt(len(esum_lcio_corr)))
    
    ### nominal LCIO guys
    esum_lcio = Esumhit_perTheta("photon-1.slcio_REC.slcio", 5000, "EcalBarrelCollection", rad, radnn)
    #esum_lcio = Esumhit_perTheta("40_deg/photon-40.slcio_REC.slcio", 2000, "EcalBarrelCollection", rad, radnn)
    
    #if esum_lcio.mean() == 0.0: continue
    lcio_y_data.append(esum_lcio.mean())
    lcio_y_err.append(esum_lcio.std() / math.sqrt(len(esum_lcio)))
    

In [ ]:
x_data = []
for i in range(30,90,5):
    rad = i*math.pi/180
    radnn = (i+5)*math.pi/180
    
    print (rad, radnn)
    x_data.append((rad + radnn)/2)

In [ ]:
for i in range(0, len(lcio_y_corrdata)):
    print (lcio_y_corrdata[i], lcio_y_correrr[i])

In [ ]:
for i in range(0, len(lcio_y_data)):
    print (lcio_y_data[i], lcio_y_err[i])

In [ ]:
plt_scatter(x_data, lcio_y_data, lcio_y_err, lcio_y_corrdata, lcio_y_correrr)

In [ ]:
a = ydata[0]
b = ydata_lcio[0]
print (a)
print(b)

In [ ]:
a, b = uncertainties.correlated_values_norm(
    [(ydata[0].n, ydata[0].s), (ydata_lcio[0].n, ydata_lcio[0].s)], [[1, 1], [1, 1]])

In [ ]:
a/b

In [ ]:
def formula(A,B,Ea, Eb):
    sigma = (A/B)*math.sqrt((Ea/A)**2 + (Eb/B)**2 - (2*Ea*Eb)/(A*B))
    return sigma

In [ ]:
A = ydata[0].nominal_value
B = ydata_lcio[0].nominal_value
Ea = ydata[0].std_dev
Eb = ydata_lcio[0].std_dev

In [ ]:
sg = formula(A,B,Ea,Eb)

In [ ]:
sg

In [ ]:
idx = np.where( (angles <= 0.698132) & (angles > 0.52389))

In [ ]:
sh = showers[idx[0]]

In [ ]:
sh.shape

In [ ]:
for i in range(30,90,5):
    rad = i*math.pi/180
    radnn = (i+5)*math.pi/180
    print ((rad + radnn)/2)
    print (rad)